In [31]:
import numpy as np
import matplotlib as plt
from math import *

In [32]:
class Plaintext:
  """Class with different computations for a plaintext object
  """
  def __init__(self, pslots, pnum_slots):
    """
    pslots: 1D array
    """
    if len(pslots) <= pnum_slots:
        concatenatedSlots = np.concatenate((pslots, np.zeros((pnum_slots - len(pslots)))))
        self.slots = np.array(concatenatedSlots)
        self.num_slots = pnum_slots
    else : 
      raise Exception("num_slots is too small.")

  def __str__(self):
    return str(self.slots)

  def add(self, other_plain):
    """Method computes the sum of two plaintext objects
  Returns:
    Plaintext sum of self and other_plain
  """

    result = self.slots + other_plain.slots
    result1 = Plaintext(result, self.num_slots)
    return result1

  def mul(self, other_plain):
    """Method computes the multiplication of two plaintext objects
  Returns:
    Plaintext product of self and other_plain
  """

    multresult = self.slots * other_plain.slots
    multresult1 = Plaintext(multresult, self.num_slots)
    return multresult1

  def rotate(self, index):
    """Method computes the rotation of an object for a specific index
  Args:
    index: number of slots to rotate by
  Returns:
    Rotated plaintext object
    ex: rotating [1,2,3,4] by 1 returns [2,3,4,1]
  """

    test = np.roll(self.slots,-1* index)
    test1 = Plaintext(test, self.num_slots)
    return test1
    

In [33]:
params = {
        'num_input_channels' : 4, # c_i
        'input_width' : 4, # w_i
        'input_height' : 4, # h_i
        'input_gap' : 2, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 16, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 2, # h_o
        'output_width' : 2, # w_o
        't_i' : 1, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 1, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}

input_image = np.array([[[i * params['num_input_channels'] * params['input_width'] + \
                          j * params['num_input_channels'] + k + 1 for k in range(params['num_input_channels'])] \
                         for j in range(params['input_width'])] for i in range(params['input_height'])])
print(input_image) # Should be (input_height, input_width, num_input_channels)
input_image_channel_0 = input_image[:, :, 0]
print(input_image_channel_0)

[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]]

 [[17 18 19 20]
  [21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]]

 [[33 34 35 36]
  [37 38 39 40]
  [41 42 43 44]
  [45 46 47 48]]

 [[49 50 51 52]
  [53 54 55 56]
  [57 58 59 60]
  [61 62 63 64]]]
[[ 1  5  9 13]
 [17 21 25 29]
 [33 37 41 45]
 [49 53 57 61]]


In [34]:
def mult_pack(A, params):
    """
    Args:
        A (numpy.ndarray): Input tensor 
        h_i : Height 
        w_i : Width 
        c_i: Number of input channels 
        params['input_gap']: Input gap
        t_i

    Returns:
        A plaintext object which has a 1-D Numpy array slots.
    """
    # Initialize the output tensor A' with zeros
    A_prime = np.zeros((params['input_gap'] * params['input_height'], params['input_gap'] * params['input_width'], params['t_i']))

    # Fill the A_prime tensor according to the given formula
    for i3 in range(params['input_gap'] * params['input_height']):  # Outer tensor height dimension
        for i4 in range(params['input_gap'] * params['input_width']):  # Outer tensor width dimension
            for i5 in range(params['t_i']):  # Channel dimension after packing
                # Map indices from A to A' based on the input gap
                orig_i3 = i3 // params['input_gap']
                orig_i4 = i4 // params['input_gap']
                orig_i5 = ((params['input_gap'] ** 2) * i5) + params['input_gap'] * (i3 % params['input_gap']) + (i4 % params['input_gap'])

                if (
                    orig_i3 < A.shape[0]
                    and orig_i4 < A.shape[1]
                    and orig_i5 < A.shape[2]
                    and orig_i5 < params['num_input_channels']
                ):
                    A_prime[i3, i4, i5] = A[orig_i3, orig_i4, orig_i5]

    return Plaintext(A_prime.flatten(), params['num_slots'])

In [35]:
p = mult_pack(input_image, params)
print(p.slots)

[1. 2. 5. ... 0. 0. 0.]


In [36]:
def SumSlots(ct_a, m, p):
    """
    SumSlots algorithm implementation using only add, multiply, and rotate.
    Args:
        ct_a (Plaintext): Input ciphertext.
        m (int): Number of added slots
        p (int): Gap
    
    """
    ct_b = [ct_a]

    for j in range(1, int(np.log2(m)) + 1):  # Inclusive loop
        rotated = ct_b[j - 1].rotate(2**(j - 1) * p)
        ct_b.append(ct_b[j - 1].add(rotated))  

    ct_c = ct_b[int(np.log2(m))]

    for j in range(0, int(np.log2(m))):  # Exclusive loop
        if (m // (2**j)) % 2 == 1:
            rotation_distance = (m // (2**(j + 1))) * 2**(j + 1) * p
            rotated = ct_b[j].rotate(rotation_distance)
            ct_c = ct_c.add(rotated)  

    return ct_c #return ciphertext

In [37]:
def weightU(params):
    weightU = np.array([[[[i * params['num_input_channels'] * params['kernel_width'] * params['num_output_channels'] + \
                              j * params['num_input_channels'] * params['num_output_channels'] + k * params['num_output_channels'] + \
                              l + 1 
                              for l in range(params['num_output_channels'])] \
                              for k in range(params['num_input_channels'])] \
                              for j in range(params['kernel_width'])] \
                              for i in range(params['kernel_height'])])
    '''
    weightU = np.zeros((params['kernel_height'], params['kernel_width'], params['num_input_channels'], params['num_output_channels']))
    count = 1
    for i in range(params['kernel_height']):
        for j in range(params['kernel_width']):
            for k in range(params['num_input_channels']):
                for l in range(params['num_output_channels']):
                    weightU[i][j][k][l] = count
                    count += 1
    print(weightU)
    '''
    return weightU

In [38]:
def mult_weight(U, i1, i2, i, params):
    """
    Creates the multiplexed shifted weight tensor U'
    
    Args:
        U: Original  tensor of shape (fh, fw, ci, co)
        i1: Index for filter height dimension
        i2: Index for filter width dimension
        i: Index for output channel dimension
        params: Dictionary containing params ki, hi, wi, ci
    
    Returns:
        Tensor U'(i1,i2,i) of shape (ki*hi, ki*wi, ti)
    """
    
    # Initialize the output tensor with zeros
    result = np.zeros((params['input_gap'] * params['input_height'], params['input_gap'] * params['input_width']), dtype=object)
    
    # Fill the tensor according to the conditions
    for i3 in range(params['input_gap'] * params['input_height']):
        for i4 in range(params['input_gap'] * params['input_width']):
            for i5 in range(params['t_i']):
            # Check all conditions (with i5=0)
                cond1 = ((params['input_gap'] ** 2) * i5) + params['input_gap'] * (i3 % params['input_gap']) + \
                            i4 % params['input_gap'] >= params['num_input_channels']
                cond2 = (i3 // params['input_gap']) - (params['kernel_height'] - 1) // 2 + i1 not in range(params['input_height'])
                cond3 = (i4 // params['input_gap']) - (params['kernel_width'] - 1) // 2 + i2 not in range(params['input_width'])
            
                if cond1 or cond2 or cond3:
                    result[i3, i4] = 0
                else:
                # Calculate indices for the original tensor U
                    u_idx = (i1, i2, ((params['input_gap'] ** 2) * i5) + \
                             params['input_gap'] * (i3 % params['input_gap']) + i4 % params['input_gap'], i)
                    result[i3, i4] = U[u_idx]
                
    return result.flatten()

In [39]:
U = weightU(params)
print(U)
print(U.shape)
mw = mult_weight(U, 0, 2, 0, params)
#print(U.shape)
#print(U)
count = 0
'''
for i in range(params['input_gap'] * params['input_height']):
    for j in range(params['input_gap'] * params['input_width']):
        # print(str(mw[i * params['input_gap'] * params['input_width'] + j])  + ", ", end="")
        print(str(mw[count]) + ", ", end="")
        count += 1
    print()
'''
test_kernel = U[:, :, 0, 0]
print(test_kernel)

[[[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16]
   [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32]
   [ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48]
   [ 49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64]]

  [[ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80]
   [ 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96]
   [ 97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112]
   [113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128]]

  [[129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144]
   [145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160]
   [161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176]
   [177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192]]]


 [[[193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208]
   [209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224]
   [225 226 227 228 229 23

In [40]:
def S_prime(params, i):
    S_prime = np.zeros((params['output_gap']*params['output_height'], params['output_gap']*params['output_width'], params['t_o']), dtype=int)

    for i3 in range(params['output_gap']*params['output_height']):
        for i4 in range(params['output_gap']*params['output_width']):
            for i5 in range(params['t_o']):
                if ((params['output_gap']**2)*i5) + params['output_gap'] * (i3 % params['output_gap']) + i4 % params['output_gap'] ==i:
                    S_prime[i3, i4, i5] = 1
                else:
                    S_prime[i3, i4, i5] = 0
    return(Plaintext(S_prime.flatten(), params['num_slots']))

In [41]:
i = 0
selecting_tensor = S_prime(params, i)
print(selecting_tensor)

[1. 0. 0. ... 0. 0. 0.]


In [42]:
def multConv(ct_aprime, U, params):
    zero_slots = np.zeros(params['num_slots'])
    ct_zero = Plaintext(zero_slots, params['num_slots'])
    ct_d = ct_zero
    ct_prime = {}
    for i1 in range(params['kernel_height']):
        for i2 in range(params['kernel_width']):
            rotation = (params['input_gap'] ** 2) * params['input_width'] * ((i1 - (params['kernel_height'] - 1) // 2)) \
                        + params['input_gap'] * ((i2 - (params['kernel_width'] - 1) // 2 ))
            ct_prime[(i1, i2)] = ct_aprime.rotate(rotation)
    for i in range(params['num_output_channels']):
        ct_b = ct_zero
        for i1 in range(params['kernel_height']):
            for i2 in range(params['kernel_width']):
                weight = mult_weight(U, i1, i2, i, params)
                weight_p = Plaintext(weight, params['num_slots'])
                ct_b = ct_b.add(ct_prime[(i1, i2)].mul(weight_p))                
        ct_c = SumSlots(ct_b, params['input_gap'], 1)
        ct_c = SumSlots(ct_c, params['input_gap'], params['input_gap'] * params['input_width'])
        ct_c = SumSlots(ct_c, params['t_i'], (params['input_gap'] ** 2) * params['input_height'] * params['input_width'])
        rot = -(((i // (params['output_gap'] ** 2)) * (params['output_gap'] ** 2) * params['output_height'] * params['output_width']) + \
                (((i % (params['output_gap']**2)) // params['output_gap']) * params['output_gap'] * params['output_width']) + \
                (i % params['output_gap']))
        ct_d = ct_d.add(ct_c.rotate(rot).mul(S_prime(params, i)))
    return ct_d

In [43]:
ct_aprime = mult_pack(input_image, params)
print(ct_aprime)
U = weightU(params)
ct_d = multConv(ct_aprime, U, params)
print(ct_d)
print(ct_d.num_slots)

'''
    [ 95432.  95632.  95832.  96032. 190396. 190840. 191284. 191728.  96232.
      96432.  96632.  96832. 192172. 192616. 193060. 193504.  97032.  97232.
      97432.  97632. 193948. 194392. 194836. 195280.  97832.  98032.  98232.
      98432. 195724. 196168. 196612. 197056. 325356. 326232. 327108. 327984.
     510522. 512052. 513582. 515112. 328860. 329736. 330612. 331488. 516642.
     518172. 519702. 521232. 332364. 333240. 334116. 334992. 522762. 524292.
     525822. 527352. 335868. 336744. 337620. 338496. 528882. 530412. 531942.
     533472.]
 '''

[1. 2. 5. ... 0. 0. 0.]
[95432.0 95632.0 95832.0 ... 0.0 0.0 0.0]
262144


'\n    [ 95432.  95632.  95832.  96032. 190396. 190840. 191284. 191728.  96232.\n      96432.  96632.  96832. 192172. 192616. 193060. 193504.  97032.  97232.\n      97432.  97632. 193948. 194392. 194836. 195280.  97832.  98032.  98232.\n      98432. 195724. 196168. 196612. 197056. 325356. 326232. 327108. 327984.\n     510522. 512052. 513582. 515112. 328860. 329736. 330612. 331488. 516642.\n     518172. 519702. 521232. 332364. 333240. 334116. 334992. 522762. 524292.\n     525822. 527352. 335868. 336744. 337620. 338496. 528882. 530412. 531942.\n     533472.]\n '

In [44]:
def zeroBorder(array):
  """Method adds a border of zeros around the array
  Args:
    n: length of the array
  Returns:
    borderArray: array with a border of zeros
  """
  if array.ndim == 2:
      n = array.shape[0]
      borderArray = np.zeros((n+2, n+2))
      borderArray[1:-1, 1:-1] = array
  elif array.ndim == 3:
      n, m, c = array.shape
      borderArray = np.zeros((n+2, m+2, c))
      borderArray[1:-1, 1:-1, :] = array
  return borderArray

In [45]:
def outputDimensions(image, stride, kernel_height, kernel_width):
    borderArray = zeroBorder(image)
    padded_height, padded_width, _ = borderArray.shape 
    outputHeight = ((padded_height - kernel_height) // stride) + 1
    outputWidth = ((padded_width - kernel_width) // stride) + 1
    return borderArray, outputHeight, outputWidth

In [46]:
def twod_naive_convolution(image, kernel, borderArray, image_height, image_width, kernel_height, kernel_width, stride, outputHeight, outputWidth):
  """Method computes the convolution of an image and a kernel with only one output channel
  Args:
    image: image as a 1D array
    kernel: kernel as a 1D array
    borderArray: image with a border of zeros
    outputImage: convoluted image
  Returns:
    The convoluted image
  """
  
  image = image.reshape(image_height, image_width)
  kernel = kernel.reshape(kernel_height, kernel_width)

  #borderArray = zeroBorder(image)
  #padded_height, padded_width = borderArray.shape 

  #print(f"The padded image is \n {borderArray}")
  #outputHeight = ((padded_height - kernel_height) // stride) + 1
  #outputWidth = ((padded_width - kernel_width) // stride) + 1
  output_Image = np.zeros((outputHeight, outputWidth))
    
  for i in range(outputHeight):
      for j in range(outputWidth):
          start_i = i * stride
          start_j = j * stride
          matrix = borderArray[start_i : start_i + kernel_height, start_j : start_j + kernel_width]
          mult = np.multiply(matrix, kernel)
          add = np.sum(mult)
          output_Image[i, j] = add
  return output_Image

def threed_naive_convolution2(image, kernel, image_height, image_width, kernel_height, kernel_width,
                              stride, num_input_channels, num_output_channels):
    """Method computes the convolution of an image and a kernel with only one output channel
    Args:
    image: image with dimensions (image_height, image_width, num_input_channels)
    kernel: kernel with dimensions (kernel_height, kernel_width. num_input_channels, num_output_channels)
    Returns:
    The convoluted image
    """
   
    """
    1 2 3 4
    5 6 7 8 
    9 10 11 12
    13 14 15 16

    stride 2 --> (0, 0) goes to (0, 0)
    (0, 2) goes to (0, 1)
    (2, 0) goes to (1, 0)
    (2, 2) goes to (1, 1)
    """
    


    
    #print(f"The padded image is \n {borderArray}")
    output_height = int(np.ceil(image_height / stride))
    output_width = int(np.ceil(image_width / stride))
    output_image = np.zeros((output_height, output_width, num_output_channels))

    for k in range(num_output_channels):
        for l in range(num_input_channels):
            for i in range(output_height):          
                for j in range(output_width):
                    for a in range(kernel_height):
                        for b in range(kernel_width):
                            start_i = stride * i - kernel_height // 2
                            start_j = stride * j - kernel_width // 2
                            if 0 <= start_i + a < image_height and 0 <= start_j + b < image_width:
                                output_image[i][j][k] += image[start_i + a][start_j + b][l] * kernel[a][b][l][k]
                                
    return output_image

In [47]:
num_input = 2
num_output = 1
image_height = 4
image_width = 4
kernel_width = 3
kernel_height = 3
stride = 2
image = np.array([1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16])
image = image.reshape((image_height, image_width, num_input))
kernel = np.array([1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9])
kernel = kernel.reshape((kernel_height, kernel_width, num_input, num_output))

threed = threed_naive_convolution2(image, kernel, image_height, image_width, kernel_height, kernel_width, stride, num_input, num_output)
test1 = threed_naive_convolution2(input_image[:, :, 1].reshape((params['input_height'], params['input_width'], 1)), U[:, :, 1, 0].reshape((params['kernel_height'], params['kernel_width'], 1, 1)), params['input_height'], params['input_width'], params['kernel_height'], params['kernel_width'], params['input_gap'], 1, 1)
print(test1)

[[[ 22576.]
  [ 45484.]]

 [[ 78552.]
  [123162.]]]


In [48]:
'''[20268.0 22576.0 33166.0 36628.0 41638.0 45484.0 27724.0 30288.0 25012.0
 27576.0 40282.0 44128.0 49522.0 53752.0 32980.0 35800.0 45618.0 49272.0
 68541.0 74022.0 77793.0 83850.0 49122.0 53160.0 53118.0 57156.0 79791.0
 85848.0 90195.0 96828.0 57390.0 61812.0 73362.0 78552.0 105549.0 113334.0
 114801.0 123162.0 70722.0 76296.0 83934.0 89508.0 121407.0 129768.0
 131811.0 140748.0 82062.0 88020.0 36524.0 40112.0 49102.0 54484.0 52966.0
 58732.0 29644.0 33488.0 43828.0 47672.0 60058.0 65824.0 64690.0 70840.0
 37460.0 41560.0]'''

'[20268.0 22576.0 33166.0 36628.0 41638.0 45484.0 27724.0 30288.0 25012.0\n 27576.0 40282.0 44128.0 49522.0 53752.0 32980.0 35800.0 45618.0 49272.0\n 68541.0 74022.0 77793.0 83850.0 49122.0 53160.0 53118.0 57156.0 79791.0\n 85848.0 90195.0 96828.0 57390.0 61812.0 73362.0 78552.0 105549.0 113334.0\n 114801.0 123162.0 70722.0 76296.0 83934.0 89508.0 121407.0 129768.0\n 131811.0 140748.0 82062.0 88020.0 36524.0 40112.0 49102.0 54484.0 52966.0\n 58732.0 29644.0 33488.0 43828.0 47672.0 60058.0 65824.0 64690.0 70840.0\n 37460.0 41560.0]'

In [49]:
'''
num_input = 2
num_output = 1
kernel_height = 3
kernel_width = 3
input_height = 3
input_width = 3
stride = 2

weightU = np.array([[[[i * num_input * kernel_width * num_output + \
                              j * num_input * num_output + k * num_output + \
                              l + 1 
                              for l in range(num_output)] \
                              for k in range(num_input)] \
                              for j in range(kernel_width)] \
                              for i in range(kernel_height)])
input_image = np.array([[[i * num_input * input_width + \
                          j * num_input + k + 1 for k in range(num_input)] \
                         for j in range(input_width)] for i in range(input_height)])
print(input_image)
print(weightU)
test = threed_naive_convolution(input_image, weightU, input_height, input_width, num_input, num_output, kernel_height, kernel_width, stride)
print(test)
'''

'\nnum_input = 2\nnum_output = 1\nkernel_height = 3\nkernel_width = 3\ninput_height = 3\ninput_width = 3\nstride = 2\n\nweightU = np.array([[[[i * num_input * kernel_width * num_output +                               j * num_input * num_output + k * num_output +                               l + 1 \n                              for l in range(num_output)]                               for k in range(num_input)]                               for j in range(kernel_width)]                               for i in range(kernel_height)])\ninput_image = np.array([[[i * num_input * input_width +                           j * num_input + k + 1 for k in range(num_input)]                          for j in range(input_width)] for i in range(input_height)])\nprint(input_image)\nprint(weightU)\ntest = threed_naive_convolution(input_image, weightU, input_height, input_width, num_input, num_output, kernel_height, kernel_width, stride)\nprint(test)\n'

In [50]:
# Check that the above output matches multPack(C)
# C = 3d-conv(input_image, U, params) with stride 2
# Implement 3d-conv with stride parameter and test with s = 2

In [51]:
outputParams = {
        'num_input_channels' : 16, # c_i
        'input_width' : 2, # w_i
        'input_height' : 2, # h_i
        'input_gap' : 4, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 16, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 2, # h_o
        'output_width' : 2, # w_o
        't_i' : 1, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 1, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}

In [52]:
print(U.shape)
print(input_image.shape)
C = threed_naive_convolution2(input_image, U, params['input_height'], params['input_width'], params['kernel_height'], params['kernel_width'], params['input_gap'], params['num_input_channels'], params['num_output_channels'],)
print(C)
print((mult_pack(C, outputParams)))

(3, 3, 4, 16)
(4, 4, 4)
[[[ 95432.  95632.  95832.  96032.  96232.  96432.  96632.  96832.
    97032.  97232.  97432.  97632.  97832.  98032.  98232.  98432.]
  [190396. 190840. 191284. 191728. 192172. 192616. 193060. 193504.
   193948. 194392. 194836. 195280. 195724. 196168. 196612. 197056.]]

 [[325356. 326232. 327108. 327984. 328860. 329736. 330612. 331488.
   332364. 333240. 334116. 334992. 335868. 336744. 337620. 338496.]
  [510522. 512052. 513582. 515112. 516642. 518172. 519702. 521232.
   522762. 524292. 525822. 527352. 528882. 530412. 531942. 533472.]]]
[95432. 95632. 95832. ...     0.     0.     0.]


In [53]:
'''
output_gap = stride * input_gap

**include stride

layer 1:
output_channels = 16
input_channels = 1
input_height = 256
input_width = 256
kernel_height = 5
kernel_width = 5
t_i = 1
t_o = 4

result:
output_height = 128
output_width = 128

layer 2:
input_channels = 16
output_channels = 32
kernel_height = 3
kernel_width = 3
t_i = 4
t_o = 8


layer 3:
input_gap = 1
output_gap = 1
input_channels = 32
output_channels = 64
kernel_height = 3
kernel_width = 3
t_i = 8
t_o = 16

layer 4:
input_gap = 1
output_gap = 1
input_channels = 64
output_channels = 128
kernel_height = 3
kernel_width = 3
t_i = 16
t_o = 32

'''

'\noutput_gap = stride * input_gap\n\n**include stride\n\nlayer 1:\noutput_channels = 16\ninput_channels = 1\ninput_height = 256\ninput_width = 256\nkernel_height = 5\nkernel_width = 5\nt_i = 1\nt_o = 4\n\nresult:\noutput_height = 128\noutput_width = 128\n\nlayer 2:\ninput_channels = 16\noutput_channels = 32\nkernel_height = 3\nkernel_width = 3\nt_i = 4\nt_o = 8\n\n\nlayer 3:\ninput_gap = 1\noutput_gap = 1\ninput_channels = 32\noutput_channels = 64\nkernel_height = 3\nkernel_width = 3\nt_i = 8\nt_o = 16\n\nlayer 4:\ninput_gap = 1\noutput_gap = 1\ninput_channels = 64\noutput_channels = 128\nkernel_height = 3\nkernel_width = 3\nt_i = 16\nt_o = 32\n\n'

In [ ]:
params_layer_one = {
        'num_input_channels' : 1, # c_i
        'input_width' : 256, # w_i
        'input_height' : 256, # h_i
        'input_gap' : 1, # k_i
        'kernel_height': 5, # f_h
        'kernel_width': 5, # f_w
        'num_output_channels' : 16, # c_o
        'output_gap' : 2, # k_o
        'stride' : 2,
        'output_height' : 128, # h_o
        'output_width' : 128, # w_o
        't_i' : 1, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 4, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}
input_image_one = np.array([[[i * params_layer_one['num_input_channels'] * params_layer_one['input_width'] + \
                          j * params_layer_one['num_input_channels'] + k + 1 for k in range(params_layer_one['num_input_channels'])] \
                         for j in range(params_layer_one['input_width'])] for i in range(params_layer_one['input_height'])])


ct_aprime_one = mult_pack(input_image_one, params_layer_one)
U_one = weightU(params_layer_one)
ct_d_one = multConv(ct_aprime_one, U_one, params_layer_one)
print(ct_d_one)

C_one = threed_naive_convolution2(input_image_one, U_one, params_layer_one['input_height'], params_layer_one['input_width'], params_layer_one['kernel_height'], params_layer_one['kernel_width'], params_layer_one['stride'], params_layer_one['num_input_channels'], params_layer_one['num_output_channels'])
#print(C_one)
print((mult_pack(C_one, params_layer_one)))

In [55]:
print(ct_d_one.num_slots)
mult_pack_C_one = mult_pack(C_one, params_layer_one)
print(mult_pack_C_one.num_slots)

print(ct_d_one.slots[: 10])
print(mult_pack_C_one.slots[: 10])

262144
262144
[794034.0 0.0 0.0 0.0 1273880.0 0.0 0.0 0.0 1290455.0 0.0]
[ 794034. 1265885. 1274075. 1282265. 1290455. 1298645. 1306835. 1315025.
 1323215. 1331405.]


In [ ]:
params_layer_two = {
        'num_input_channels' : 16, # c_i
        'input_width' : 64, # w_i
        'input_height' : 64, # h_i
        'input_gap' : 2, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 32, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 32, # h_o
        'output_width' : 32, # w_o
        't_i' : 4, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 8, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}
input_image_two = np.array([[[i * params_layer_two['num_input_channels'] * params_layer_two['input_width'] + \
                          j * params_layer_two['num_input_channels'] + k + 1 for k in range(params_layer_two['num_input_channels'])] \
                         for j in range(params_layer_two['input_width'])] for i in range(params_layer_two['input_height'])])


ct_aprime_two = mult_pack(input_image_two, params_layer_two)
print(ct_aprime_two.num_slots)
U_two = weightU(params_layer_two)
print(U_two.shape)
ct_d_two = multConv(ct_aprime_two, U_two, params_layer_two)
print(ct_d_two)

C_two = threed_naive_convolution2(input_image_two, U_two, params_layer_two['input_height'], params_layer_two['input_width'], params_layer_two['kernel_height'], params_layer_two['kernel_width'], params_layer_two['input_gap'], params_layer_two['num_input_channels'], params_layer_two['num_output_channels'])
#print(C_two.shape)
print((mult_pack(C_two, params_layer_two)))

262144
(3, 3, 16, 32)


In [58]:
params_layer_three = {
        'num_input_channels' : 32, # c_i
        'input_width' : 16, # w_i
        'input_height' : 16, # h_i
        'input_gap' : 4, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 64, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 16, # h_o
        'output_width' : 16, # w_o
        't_i' : 8, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 16, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}
input_image_three = np.array([[[i * params_layer_three['num_input_channels'] * params_layer_three['input_width'] + \
                          j * params_layer_three['num_input_channels'] + k + 1 for k in range(params_layer_three['num_input_channels'])] \
                         for j in range(params_layer_three['input_width'])] for i in range(params_layer_three['input_height'])])


ct_aprime_three = mult_pack(input_image_three, params_layer_three)
U_three = weightU(params_layer_three)
ct_d_three = multConv(ct_aprime_three, U_three, params_layer_three)
print(ct_d_three)

C_three = threed_naive_convolution2(input_image_three, U_three, params_layer_three['input_height'], params_layer_three['input_width'], params_layer_three['kernel_height'], params_layer_three['kernel_width'], params_layer_three['input_gap'], params_layer_three['num_input_channels'], params_layer_three['num_output_channels'])
print((mult_pack(C_three, params_layer_three)))

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
[5.93899584e+08 5.94490432e+08 0.00000000e+00 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [59]:
params_layer_four = {
        'num_input_channels' : 32, # c_i
        'input_width' : 16, # w_i
        'input_height' : 16, # h_i
        'input_gap' : 4, # k_i
        'kernel_height': 3, # f_h
        'kernel_width': 3, # f_w
        'num_output_channels' : 64, # c_o
        'output_gap' : 4, # k_o
        'output_height' : 16, # h_o
        'output_width' : 16, # w_o
        't_i' : 8, # t_i: c_i/k_i^2, number of squares to represent all input channels
        't_o' : 16, # t_o: c_o/k_o^2, number of squares to represent all input channels
        'num_slots' : 2**18
}
input_image_two = np.array([[[i * params_layer_two['num_input_channels'] * params_layer_two['input_width'] + \
                          j * params_layer_two['num_input_channels'] + k + 1 for k in range(params_layer_two['num_input_channels'])] \
                         for j in range(params_layer_two['input_width'])] for i in range(params_layer_two['input_height'])])


ct_aprime_two = mult_pack(input_image_two, params_layer_two)
U_two = weightU(params_layer_two)
ct_d_two = multConv(ct_aprime_two, U_two, params_layer_two)
print(ct_d_two)

C_two = threed_naive_convolution2(input_image_two, U_two, params_layer_two['input_height'], params_layer_two['input_width'], params_layer_two['kernel_height'], params_layer_two['kernel_width'], params_layer_two['input_gap'], params_layer_two['num_input_channels'], params_layer_two['num_output_channels'])
print((mult_pack(C_two, params_layer_two)))

[25155440.0 30271992.0 0.0 ... 0.0 0.0 0.0]
[1.37399328e+08 1.37534624e+08 1.37669920e+08 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
